In [1]:
import urllib2
import re
import lxml.html
from __future__ import division
from datetime import datetime
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
import string
import nltk, re, pprint
from nltk import word_tokenize
import pymongo

In [2]:
def download(url, user_agent='liter', num_retries=2):
    #print('Downloading', url)
    headers = {'User-agent': user_agent}
    request = urllib2.Request(url, headers=headers)
    try:
        html = urllib2.urlopen(request).read()
    except urllib2.URLError as e:
        print 'Download error:', e.reason
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                #recursively retry 5xx HTTP errors
                return download(url, user_agent, num_retries-1)   
    return html

In [4]:
#def genre_urls(genre_url):
    #deprecate for db-driven get_urls_dic below
    #genre = 'fetish-stories'
    #html = download(genre_url)
    #tree = lxml.html.fromstring(html)
    #story_urls = tree.xpath('//div[@class="b-sl-item-r w-34t"]/h3/a/@href')
    #story_titles = tree.xpath('//div[@class="b-sl-item-r w-34t"]/h3/a/text()')
    #story_descs = [re.sub(u'\xa0\u2014\xa0','',desc) for desc in tree.xpath('//div[@class="b-sl-item-r w-34t"]/span[@class="b-sli-description p-57u"]/text()')]
    #story_author_names = tree.xpath('//div[@class="b-sl-item-r w-34t"]/span[@class="b-sli-meta"]/span[@class="b-sli-author"]/a/text()')
    #story_author_ids = [re.match('.*uid=(\w+)&',id).group(1) for id in tree.xpath('//div[@class="b-sl-item-r w-34t"]/span[@class="b-sli-meta"]/span[@class="b-sli-author"]/a/@href')]
    #story_dates = [datetime.strptime(date, '%m/%d/%y') for date in tree.xpath('//div[@class="b-sl-item-r w-34t"]/span[@class="b-sli-meta"]/span[@class="b-sli-date"]/text()')]
    #story_ratings = [float(rating) for rating in tree.xpath('//div[@class="b-sl-item-r w-34t"]/span[@class="b-sli-meta"]/span[@class="b-sli-rating"]/text()')]
    #return story_urls

In [19]:
def genre_urls_dic(genre_url):
    # given a genre, and a seed url, output a dictionary of all stats/stories/urls for a given genre
    genre = re.findall('^.+/c/(.+)',genre_url)
    url = genre_url+'/1-page'
    html = download(url)
    tree = lxml.html.fromstring(html)
    pages = int(tree.xpath('//div[@class="b-pager-pages"]/form/select/option/text()')[-1])
    
    genre_data = []
    
    for i in range(1,pages+1):
        page_url = 'https://www.literotica.com/c/fetish-stories/%s-page' % (i)
        # harvest data
        ## get list of story elements
        story_roots = tree.xpath('//div[@class="b-sl-item-r w-34t"]')
        ## loop through stories to create dic of associated data
        for root in story_roots:
            story_data = {}
            story_data['genre'] = genre
            story_data['url'] = root.xpath('./h3/a/@href')[0]
            story_data['story_id'] = re.findall('^.+/s/(.+)|$', story_data['url'])[0]
            story_data['title'] = root.xpath('./h3/a/text()')[0]
            story_data['desc'] = re.sub(u'\xa0\u2014\xa0','',root.xpath('./span[@class="b-sli-description p-57u"]/text()')[0])
            story_data['author_name'] = root.xpath('./span[@class="b-sli-meta"]/span[@class="b-sli-author"]/a/text()')[0]
            story_data['author_id'] = re.findall('.*uid=(\w+)&|$',root.xpath('./span[@class="b-sli-meta"]/span[@class="b-sli-author"]/a/@href')[0])[0]
            story_data['created_date'] = datetime.strptime(root.xpath('./span[@class="b-sli-meta"]/span[@class="b-sli-date"]/text()')[0], '%m/%d/%y')
            try:
                story_data['rating'] = float(root.xpath('./span[@class="b-sli-meta"]/span[@class="b-sli-rating"]/text()')[0])
            except (IndexError, ValueError):
                story_data['rating'] = None
            
            genre_data.append(story_data)
        
        
        

    
    return genre_data


In [65]:
fetish_url = 'https://www.literotica.com/c/fetish-stories'
html = download(fetish_url+'/1-page')
tree = lxml.html.fromstring(html)
story_urls = tree.xpath('//div[@class="b-sl-item-r w-34t"]/h3/a/@href')
len(story_urls)
#story_titles = tree.xpath('//div[@class="b-sl-item-r w-34t"[contains("%s")]]/h3/a/text()' % (story_urls[1]))
root = tree.xpath('//div[@class="b-sl-item-r w-34t"]')
title = root[8].xpath('./h3/a/text()')
try:
    rating = float(root[8].xpath('./span[@class="b-sli-meta"]/span[@class="b-sli-rating"]/text()'))
except:
    rating = ''
print title, rating




['1,200 pound orgy'] 


In [20]:
xtest = genre_urls_dic('https://www.literotica.com/c/fetish-stories')

In [35]:
xtest[:15]
summy = 0
for story in xtest:
    summy += story['rating']
print(summy)

TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'

In [45]:
def get_story(story_url):
    html = download(story_url)
    tree = lxml.html.fromstring(html)
    pages = int(re.match('(\d+)',tree.xpath('//div[@class="b-pager-pages"]/span[@class="b-pager-caption-t r-d45"]/text()')[0]).group(0))
    text = tree.xpath('//div[@class="b-story-body-x x-r15"]/div/p/text()')
    
    story_text = []
    story_text += text    
    
    if pages > 1:
        for i in range(1,pages):
            page_url = story_url + '?page=%s' % (i+1)
            page_html = download(page_url)
            page_tree = lxml.html.fromstring(page_html)
            page_text = page_tree.xpath('//div[@class="b-story-body-x x-r15"]/div/p/text()')
            story_text += page_text

    return story_text


In [41]:
def clean_story(raw_story):
    tokens = word_tokenize(" ".join(raw_story))
    tagged = nltk.tag.pos_tag(tokens)
    non_nnp = [word.lower() for word,tag in tagged if tag != 'NNP' and tag != 'NNPS']

    content = [word for word in non_nnp if word not in stopwords and word not in punc]
    lemmad = [lemma.lemmatize(word) for word in content]
    
    return lemmad

In [ ]:
fetish_url = 'https://www.literotica.com/c/fetish-stories/1-page'
stories = []
for i in range(1,78):
    test_url = 'https://www.literotica.com/c/fetish-stories/%s-page' % (i)
    stories.extend(genre_urls(test_url))

len(stories)


In [29]:
tree = lxml.html.fromstring(download(fetish_url))
pagex = tree.xpath('//div[@class="b-pager-pages"]/form/select/option/text()')
page_max = int(pagex[-1])
page_max

('Downloading', 'https://www.literotica.com/c/fetish-stories/1-page')


77

In [33]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(["n't","'s","''","``","--",'...',"'re","'m",'would','woman','man','dick','cock','pussy','cunt','hand','one','two','as','get','said','thought','like'])
punc = string.punctuation
lemma = WordNetLemmatizer()
p_stemmer = PorterStemmer()

In [44]:
get_story(stories[0])

('Downloading', 'https://www.literotica.com/s/are-they-nice')


In [1]:
texts = []
prog = 0
totes = len(stories)
for url in stories:
    raw_story = get_story(url)
    texts.append(clean_story(raw_story))
    prog += 1
    print(str(prog) + " of " + str(totes) + "(" + str(round(prog/totes,3)*100) + "%)")
    


In [59]:
import pickle
totals = 0
for text in texts:
    totals += len(text)
print totals, len(texts), totals/len(texts)
output = open('fet-partial.pkl', 'wb')
pickle.dump(texts, output)
output.close()

6151326 3956 1554.93579373


In [3]:
import pickle
pkl_file = open('fet-partial.pkl', 'rb')
fet = pickle.load(pkl_file)
pkl_file.close()

In [6]:
len(fet)
fet1 = fet[:2000]
fet2 = fet[2000:]
output = open('fet-partial.pkl','wb')
pickle.dump(fet,output)
output.close()


In [15]:
import gensim
from gensim import corpora, models
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [16]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word = dictionary, passes = 10)

In [17]:
topics = ldamodel.print_topics(num_topics=30, num_words=8)
for i in topics:
    print i

(0, u'0.012*"would" + 0.008*"woman" + 0.007*"cock" + 0.007*"man" + 0.006*"said" + 0.006*"hand" + 0.005*"back" + 0.005*"thought"')
(1, u'0.028*"leg" + 0.011*"hand" + 0.009*"bare" + 0.009*"said" + 0.009*"latex" + 0.008*"panty" + 0.007*"would" + 0.006*"could"')
(2, u'0.012*"said" + 0.011*"could" + 0.009*"cock" + 0.009*"back" + 0.007*"like" + 0.007*"would" + 0.006*"time" + 0.006*"hand"')
(3, u'0.022*"..." + 0.009*"would" + 0.008*"cock" + 0.007*"sow" + 0.006*"like" + 0.006*"back" + 0.006*"hand" + 0.005*"little"')
(4, u'0.009*"could" + 0.007*"would" + 0.006*"one" + 0.006*"get" + 0.005*"like" + 0.005*"..." + 0.005*"time" + 0.005*"little"')
(5, u'0.002*"hammock" + 0.002*"warehouse" + 0.001*"lycra" + 0.001*"squelch" + 0.001*"fence" + 0.001*"cock" + 0.001*"back" + 0.001*"taped"')
(6, u'0.004*"would" + 0.004*"one" + 0.003*"like" + 0.003*"looked" + 0.003*"could" + 0.003*"office" + 0.002*"book" + 0.002*"mind"')
(7, u'0.007*"would" + 0.007*"could" + 0.006*"like" + 0.006*"one" + 0.006*"back" + 0.006*